In [1]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [2]:
men = pd.DataFrame({'id_man': [0, 1, 2, 3], 
                    'name': ['Вася', 'Петя', 'Миша', 'Вася'], 
                    'age': [30, 50, 40, 50]})
women = pd.DataFrame({'id_woman': [0, 1, 2, 3], 
                        'name': ['Маша', 'Аня', 'Аня', 'Лена'], 
                        'age': [25, 51, 38, 38]})
children = pd.DataFrame({'id_child': [0, 1, 2, 3, 4, 5], 
                        'name': ['Миша', 'Аня', 'Лиза', 'Лена', 'Боря', 'Рома'], 
                        'gender': ['b', 'g', 'g', 'g', 'b', 'b'],
                        'age': [5, 11, 14, 8, 12, 11],
                        'id_man': [0, 1, 1, 2, 2, 2],
                        'id_woman': [0, 1, 1, 2, 2, 2]})

In [3]:
emp = pd.DataFrame({
    'empno': [7369, 7499, 7521, 7566, 7654, 7698, 7782, 7788, 7839, 7844, 7876, 7900, 7902, 7934],
    'ename': ['SMITH', 'ALLEN', 'WARD', 'JONES', 'MARTIN', 'BLAKE', 'CLARK', 'SCOTT', 'KING', 'TURNER', 'ADAMS', 'JAMES', 'FORD', 'MILLER'],
    'job': ['CLERK', 'SALESMAN', 'SALESMAN', 'MANAGER', 'SALESMAN', 'MANAGER', 'MANAGER', 'ANALYST', 'PRESIDENT', 'SALESMAN', 'CLERK', 'CLERK', 'ANALYST', 'CLERK'],
    'mgr': [7902, 7698, 7698, 7839, 7698, 7839, 7839, 7566, np.nan, 7698, 7788, 7698, 7566, 7782],
    'hiredate': ['17-DEC-1980', '20-FEB-1981', '22-FEB-1981', '02-APR-1981', '28-SEP-1981', '01-MAY-1981', '09-JUN-1981', '09-DEC-1982', '17-NOV-1981', '08-SEP-1981', '12-JAN-1983', '03-DEC-1981', '03-DEC-1981', '23-JAN-1982'],
    'sal': [800, 1600, 1250, 2975, 1250, 2850, 2450, 3000, 5000, 1500, 1100, 950, 3000, 1300],
    'comm': [np.nan, 300, 500, np.nan, 1400, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'deptno': [20, 30, 30, 20, 30, 30, 10, 20, 10, 30, 20, 30, 20, 10]
})
emp['hiredate'] = pd.to_datetime(emp['hiredate'])
'sal - зарплата, comm - комиссия'
dept = pd.DataFrame({
    'deptno': [10, 20, 30, 40],
    'dname': ['ACCOUNTING', 'RESEARCH', 'SALES', 'OPERATIONS'],
    'loc': ['NEW YORK', 'DALLAS', 'CHICAGO', 'BOSTON']
})

Код из занятия Карпова

In [141]:
# В таблице user_actions сохраняются действия покупателей в интернет магазине, структура таблицы

# user_id    - id пользователя
# product_id - id товара
# action     - действие, просмотр товара, сохранение в корзину, покупка
# time       - время совершения события
# date       - дата события

# Для каждого дня рассчитайте, какой процент просмотров товаров завершился покупкой

In [2]:
all_user_ids = np.arange(1, 1001)
all_product_ids = np.arange(1, 101)

In [3]:
n = 10000

In [4]:
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

In [5]:
start_date = pd.to_datetime('2022-01-01')

In [6]:
times = pd.date_range(start_date, periods=n, freq='1min')

In [7]:
user_actions = pd.DataFrame({'user_id': user_ids, 
                             'product_id': product_ids, 
                             'time': times})

In [8]:
user_actions['action'] = 'view'

In [9]:
user_actions.head()

,user_id,product_id,time,action
0,191,18,2022-01-01 00:00:00,view
1,960,44,2022-01-01 00:01:00,view
2,401,70,2022-01-01 00:02:00,view
3,418,36,2022-01-01 00:03:00,view
4,769,92,2022-01-01 00:04:00,view


In [10]:
def generate_funel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1)[0]:
        df = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(5, unit='s'), 
                           'action'    : 'add to cart'}, index=[0])
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(10, unit='s'), 
                           'action'    : 'purchase'}, index=[0])
    
            df = df.append(df_purchase)
    return df

In [11]:
to_cart_df = pd.DataFrame()

In [12]:
for index, row in user_actions.iterrows():
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    to_cart_df = to_cart_df.append(user_df)
    

In [13]:
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')

In [14]:
user_actions['date'] = user_actions.time.dt.date

In [15]:
user_actions.head()

,user_id,product_id,time,action,date
0,191,18,2022-01-01 00:00:00,view,2022-01-01
1,960,44,2022-01-01 00:01:00,view,2022-01-01
0,960,44,2022-01-01 00:01:05,add to cart,2022-01-01
2,401,70,2022-01-01 00:02:00,view,2022-01-01
3,418,36,2022-01-01 00:03:00,view,2022-01-01


In [16]:
q = """SELECT date, 
              views, 
              carts, 
              purchases, 
              100 * purchases / views as purchase_percantage 
       FROM (
           SELECT date, 
            count(case when action = 'view'        then 1 else NULL end) as views, 
            count(case when action = 'add to cart' then 1 else NULL end) as carts, 
            count(case when action = 'purchase'    then 1 else NULL end) as purchases 
           FROM user_actions 
           GROUP BY date);"""


In [17]:
sqldf(q)

,date,views,carts,purchases,purchase_percantage
0,2022-01-01,1440,258,107,7
1,2022-01-02,1440,315,121,8
2,2022-01-03,1440,279,119,8
3,2022-01-04,1440,299,127,8
4,2022-01-05,1440,228,86,5
5,2022-01-06,1440,288,106,7
6,2022-01-07,1360,283,110,8


Все ограничения SQLite, например: 
- не реализуется правое и полное внешнее соединения;
- не знает ALL и ANY.

Выполняет только запросы и не может: обновление, вставку и изменение таблиц.

In [2]:
men = pd.DataFrame({'id_man': [0, 1, 2, 3], 
                    'name': ['Вася', 'Петя', 'Миша', 'Вася'], 
                    'age': [30, 50, 40, 50]})
women = pd.DataFrame({'id_woman': [0, 1, 2, 3], 
                        'name': ['Маша', 'Аня', 'Аня', 'Лена'], 
                        'age': [25, 51, 38, 38]})
children = pd.DataFrame({'id_child': [0, 1, 2, 3, 4, 5], 
                        'name': ['Дима', 'Аня', 'Лиза', 'Лена', 'Боря', 'Рома'], 
                        'gender': ['b', 'g', 'g', 'g', 'b', 'b'],
                        'age': [5, 11, 14, 8, 12, 11],
                        'id_man': [0, 1, 1, 2, 2, 2],
                        'id_woman': [0, 1, 1, 2, 2, 2]})

In [5]:
q = """SELECT c.name, c.age, m.name father, m.age f_age, w.name mother, w.age m_age 
       FROM children c 
       JOIN men m ON c.id_man = m.id_man
       JOIN women w ON c.id_woman = w.id_woman;"""
sqldf(q)

,name,age,father,f_age,mother,m_age
0,Дима,5,Вася,30,Маша,25
1,Аня,11,Петя,50,Аня,51
2,Лиза,14,Петя,50,Аня,51
3,Лена,8,Миша,40,Аня,38
4,Боря,12,Миша,40,Аня,38
5,Рома,11,Миша,40,Аня,38


In [6]:
"""Имена девочек, которые не встречаются среди женщин"""
q = """SELECT DISTINCT name 
       FROM children
       WHERE gender = 'g'
       EXCEPT
       SELECT DISTINCT name 
       FROM women;"""
sqldf(q)

,name
0,Лиза


In [7]:
"""Имена девочек, которые не встречаются среди женщин"""
q = """SELECT DISTINCT name 
       FROM children
       WHERE gender = 'g'
       AND name NOT IN (SELECT DISTINCT name FROM women);"""
sqldf(q)

,name
0,Лиза


In [9]:
q = "SELECT DISTINCT name FROM children WHERE gender = 'g'"
sqldf(q)

,name
0,Маша
1,Аня
2,Лена


In [11]:
'Имена детей, у которых отцы старше 40'
q = """SELECT c.name 
       FROM children c 
       JOIN men m ON c.id_man = m.id_man
       WHERE m.age > 40;"""
sqldf(q)

,name
0,Аня
1,Лиза


In [14]:
'Имена детей, у которых отцы старше 40'
q = """SELECT c.name 
       FROM children c
       WHERE id_man IN (SELECT id_man
                        FROM men m
                        WHERE m.age > 40);"""
sqldf(q)

,name
0,Аня
1,Лиза


In [16]:
'имя самого старшего из мужчин'
q = '''SELECT name
        FROM men
        WHERE age = (SELECT MAX(age) FROM men);'''
sqldf(q)

,name
0,Петя
1,Вася


In [11]:
'вывести имена мужчин, чей возраст отличается от среднего более, чем на 10 лет'
q = '''SELECT name
        FROM men
        WHERE ABS(age - (SELECT AVG(age) FROM men)) > 10;'''
sqldf(q)

,name
0,Вася


In [ ]:
'вывести имена детей, чей возраст меньше самого большого из средних возрастов и мальчиков, и девочек'
q = '''SELECT name, age
       FROM children
       WHERE age < ANY(SELECT AVG(age) 
                       FROM children
                       GROUP BY gender);'''
'в sqlite нет ANY и ALL'

In [20]:
'вывести имена детей и на сколько их возраст отличается от среднего'
q = '''SELECT name, age - (SELECT AVG(age) FROM children)
        FROM children;'''
sqldf(q)

,name,age - (SELECT AVG(age) FROM children)
0,Дима,-5.166667
1,Аня,0.833333
2,Лиза,3.833333
3,Лена,-2.166667
4,Боря,1.833333
5,Рома,0.833333


In [9]:
'вывести имена самых младших детей обоих полов'
q = '''SELECT name, age
       FROM children
       WHERE (gender, age) IN (SELECT gender, MIN(age)
                               FROM children
                               GROUP BY gender);'''
sqldf(q)

,name,age
0,Миша,5
1,Лена,8


In [5]:
'вывести имя старшего ребенка'
q = '''SELECT name, age
       FROM children
       WHERE age = (SELECT MAX(age) FROM children);'''
sqldf(q)

,name,age
0,Лиза,14


In [10]:
'на сколько каждая девочка старше самого младшего ребенка'
q = '''SELECT name, age - (SELECT MIN(age) FROM children) 'разница'
       FROM children
       WHERE gender = 'g';'''
sqldf(q)

,name,разница
0,Аня,6
1,Лиза,9
2,Лена,3


In [5]:
'имена родителей Миши'
q = '''SELECT m.name, w.name
       FROM children c
       JOIN men m ON m.id_man = c.id_man
       JOIN women w ON w.id_woman = c.id_woman
       WHERE c.name = 'Миша';'''
sqldf(q)

,name,name
0,Вася,Маша


In [11]:
'имена девочек, имеющих, по крайней мере, одну сестру (каждый отец в одной семье детей имеет)'
q = '''SELECT name
       FROM children
       WHERE gender = 'g'
       AND id_man IN (SELECT id_man
                       FROM children
                       WHERE gender = 'g'
                       GROUP BY id_man
                       HAVING COUNT(*) > 1);'''
sqldf(q)

,name
0,Аня
1,Лиза
2,Лена


In [20]:
'id мужчин имеющих по крайней мере одну дочь'
q = '''SELECT id_man
       FROM children
       WHERE gender = 'g'
       GROUP BY id_man
       HAVING COUNT(*) >= 1;'''
sqldf(q)

,id_man
0,1
1,2


In [3]:
children

,id_child,name,gender,age,id_man,id_woman
0,0,Миша,b,5,0,0
1,1,Аня,g,11,1,1
2,2,Лиза,g,14,1,1
3,3,Лена,g,8,2,2
4,4,Боря,b,12,2,2
5,5,Рома,b,11,2,2


In [7]:
'имена детей, у которых мама - Аня, а папа - Петя'
q = '''SELECT c.name
       FROM children c JOIN men m ON c.id_man = m.id_man
       WHERE m.name = 'Петя'
       
       INTERSECT
       
       SELECT c.name
       FROM children c JOIN women w ON c.id_woman = w.id_woman
       WHERE w.name = 'Аня';'''
sqldf(q)

,name
0,Аня
1,Лиза


In [3]:
q = '''SELECT c.name
       FROM children c 
       JOIN men m ON c.id_man = m.id_man
       JOIN women w ON c.id_woman = w.id_woman
       WHERE m.name = 'Петя' and w.name = 'Аня';'''
sqldf(q)

,name
0,Аня
1,Лиза


In [4]:
q = '''SELECT c.name
       FROM children c JOIN men m ON c.id_man = m.id_man
       WHERE m.name = 'Петя'
       AND c.name IN(SELECT c.name
                     FROM children c JOIN women w ON c.id_woman = w.id_woman
                     WHERE w.name = 'Аня');'''
sqldf(q)

,name
0,Аня
1,Лиза


In [5]:
q = """SELECT c.name, c.age, m.name father, m.age f_age, w.name mother, w.age m_age 
       FROM children c 
       JOIN men m ON c.id_man = m.id_man
       JOIN women w ON c.id_woman = w.id_woman;"""
sqldf(q)

,name,age,father,f_age,mother,m_age
0,Миша,5,Вася,30,Маша,25
1,Аня,11,Петя,50,Аня,51
2,Лиза,14,Петя,50,Аня,51
3,Лена,8,Миша,40,Аня,38
4,Боря,12,Миша,40,Аня,38
5,Рома,11,Миша,40,Аня,38


In [10]:
'Средний возраст матерей и отцов девочек'
q = """SELECT AVG(m.age), AVG(w.age) 
       FROM children c 
       JOIN men m ON c.id_man = m.id_man
       JOIN women w ON c.id_woman = w.id_woman
       WHERE c.gender = 'g';"""
sqldf(q)

,AVG(m.age),AVG(w.age)
0,46.666667,46.666667


In [15]:
'Количество мужчин имеющих одного ребенка'
q = """SELECT COUNT(*) 
       FROM(SELECT id_man 
            FROM children
            GROUP BY id_man
            HAVING COUNT(*) = 1) tmp;"""
sqldf(q)

,COUNT(*)
0,1


In [17]:
'Количество мужчин имеющих одного ребенка'
q = """SELECT COUNT(*) FROM children
        WHERE id_man in (SELECT id_man FROM children
                         GROUP BY id_man
                         HAVING COUNT(*) = 1);"""
sqldf(q)

,COUNT(*)
0,1


In [4]:
'Имена мужчин и женщин, начинающиеся на "М"'
q = """
        SELECT name FROM men WHERE name LIKE 'М%'
        UNION
        SELECT name FROM women WHERE name LIKE 'М%';"""
sqldf(q)

,name
0,Маша
1,Миша


In [10]:
'вывести id_man с наибольшим и наименьшим количеством детей'
q = """
        WITH counter AS (
                SELECT 
                    COUNT(id_man) cnt 
                FROM 
                    children 
                GROUP BY 
                    id_man)

        SELECT
            id_man, 
            COUNT(id_man)
        FROM 
            children 
        GROUP BY 
            id_man 
        HAVING 
            COUNT(id_man) IN (
                    (SELECT MAX(cnt) FROM counter), 
                    (SELECT MIN(cnt) FROM counter));"""
sqldf(q)

,id_man,COUNT(id_man)
0,0,1
1,2,3


In [ ]:
WITH counter AS (SELECT COUNT(id_man) cnt FROM children GROUP BY id_man)
SELECT id_man FROM children GROUP BY id_man 
HAVING COUNT(id_man) IN (
            (SELECT MAX(cnt) FROM counter), 
            (SELECT MIN(cnt) FROM counter));

## Молинаро - SQL. Сборник рецептов

In [32]:
emp = pd.DataFrame({
    'empno': [7369, 7499, 7521, 7566, 7654, 7698, 7782, 7788, 7839, 7844, 7876, 7900, 7902, 7934],
    'ename': ['SMITH', 'ALLEN', 'WARD', 'JONES', 'MARTIN', 'BLAKE', 'CLARK', 'SCOTT', 'KING', 'TURNER', 'ADAMS', 'JAMES', 'FORD', 'MILLER'],
    'job': ['CLERK', 'SALESMAN', 'SALESMAN', 'MANAGER', 'SALESMAN', 'MANAGER', 'MANAGER', 'ANALYST', 'PRESIDENT', 'SALESMAN', 'CLERK', 'CLERK', 'ANALYST', 'CLERK'],
    'mgr': [7902, 7698, 7698, 7839, 7698, 7839, 7839, 7566, np.nan, 7698, 7788, 7698, 7566, 7782],
    'hiredate': ['17-DEC-1980', '20-FEB-1981', '22-FEB-1981', '02-APR-1981', '28-SEP-1981', '01-MAY-1981', '09-JUN-1981', '09-DEC-1982', '17-NOV-1981', '08-SEP-1981', '12-JAN-1983', '03-DEC-1981', '03-DEC-1981', '23-JAN-1982'],
    'sal': [800, 1600, 1250, 2975, 1250, 2850, 2450, 3000, 5000, 1500, 1100, 950, 3000, 1300],
    'comm': [np.nan, 300, 500, np.nan, 1400, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'deptno': [20, 30, 30, 20, 30, 30, 10, 20, 10, 30, 20, 30, 20, 10]
})
emp['hiredate'] = pd.to_datetime(emp['hiredate'])
# sal - зарплата, comm - комиссия
dept = pd.DataFrame({
    'deptno': [10, 20, 30, 40],
    'dname': ['ACCOUNTING', 'RESEARCH', 'SALES', 'OPERATIONS'],
    'loc': ['NEW YORK', 'DALLAS', 'CHICAGO', 'BOSTON']
})

In [4]:
emp

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7369,SMITH,CLERK,7902.0,1980-12-17,800,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500,NaN,30


In [5]:
dept

,deptno,dname,loc
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON


In [10]:
q = """select a.ename, b.ename mgrname
from emp a join emp b
on (a.mgr = b.empno);"""
sqldf(q)

,ename,mgrname
0,SMITH,FORD
1,ALLEN,BLAKE
2,WARD,BLAKE
3,JONES,KING
4,MARTIN,BLAKE
5,BLAKE,KING
6,CLARK,KING
7,SCOTT,JONES
8,TURNER,BLAKE
9,ADAMS,SCOTT


In [25]:
'типы сотрудников, которых нанимали и в 1980, и 1982 году'
q = """
SELECT job
FROM emp
WHERE hiredate BETWEEN '1980-01-01' and '1980-12-31'
INTERSECT
SELECT job
FROM emp
WHERE hiredate BETWEEN '1982-01-01' and '1982-12-31';
"""
sqldf(q)

,job
0,CLERK


In [21]:
'типы сотрудников, которых нанимали и в 1980, и 1982 году'
q = """
SELECT c1.id_child, c1.name, COUNT(c2.id_child) AS count
FROM children c1
    LEFT JOIN children c2 
        ON c1.id_woman == c2.id_woman
            AND c1.id_child != c2.id_child
GROUP BY c1.id_child;
"""
sqldf(q)

,id_child,name,count
0,0,Миша,0
1,1,Аня,1
2,2,Лиза,1
3,3,Лена,2
4,4,Боря,2
5,5,Рома,2


In [6]:
children

,id_child,name,gender,age,id_man,id_woman
0,0,Миша,b,5,0,0
1,1,Аня,g,11,1,1
2,2,Лиза,g,14,1,1
3,3,Лена,g,8,2,2
4,4,Боря,b,12,2,2
5,5,Рома,b,11,2,2


In [4]:
men

,id_man,name,age
0,0,Вася,30
1,1,Петя,50
2,2,Миша,40
3,3,Вася,50


In [5]:
women

,id_woman,name,age
0,0,Маша,25
1,1,Аня,51
2,2,Аня,38
3,3,Лена,38


In [5]:
emp

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7369,SMITH,CLERK,7902.0,1980-12-17,800,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500,NaN,30


In [6]:
dept

,deptno,dname,loc
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON
